**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
import numpy as np
import os

software = '/home/mario/DockM8/software'
protein_file = '/home/mario/holiday/igf1r/receptor_protoss_prepared.pdb'
ref_file = '/home/mario/holiday/igf1r/crystal_ligand_protoss.sdf'
docking_library = '/home/mario/holiday/igf1r/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4
ncpus = int(os.cpu_count()/2)
#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
pocket = GetPocket(ref_file, protein_file, 8)

In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver', ncpus)

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
fetch_poses_splitted(w_dir, n_poses, split_files_folder=w_dir+'/temp/split_final_library')

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
    cluster_futures(f'{metric}', 'KMedoids', w_dir, protein_file, all_poses, ncpus)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [4]:
for metric in clustering_metrics:
    rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 1, ncpus)


[2023-Mar-14 08:33:49]: Rescoring with CHEMPLP complete in 73.3911!

[2023-Mar-14 08:33:49]: Rescoring with RFScoreVS

[2023-Mar-14 08:35:29]: Rescoring with RF-Score-VS complete in 100.8085!
Splitting SDF file spyRMSD_clustered.sdf ...


Splitting files: 100%|██████████| 24/24 [00:03<00:00,  7.95it/s]


Split docking library into 24 files each containing 1066 compounds


Rescoring with LinF9: 100%|██████████| 24/24 [00:05<00:00,  4.25file/s]



[2023-Mar-14 08:35:46]: Rescoring with LinF9 complete in 16.9255!

[2023-Mar-14 08:35:46]: Combining all score for /home/mario/holiday/igf1r/temp/rescoring_spyRMSD_clustered


...

Split docking library into 25 files each containing 371 compounds


Rescoring with LinF9: 100%|██████████| 25/25 [00:02<00:00,  9.07file/s]



[2023-Mar-14 09:03:24]: Rescoring with LinF9 complete in 7.3387!

[2023-Mar-14 09:03:24]: Combining all score for /home/mario/holiday/igf1r/temp/rescoring_bestpose_PLANTS_clustered


Combining scores: 100%|██████████| 4/4 [00:00<00:00, 453.39files/s]


[2023-Mar-14 09:03:24]: Rescoring complete in 178.5324!


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [5]:
apply_consensus_methods(w_dir, clustering_metrics)

The folder: /home/mario/holiday/igf1r/temp/ranking was created
The folder: /home/mario/holiday/igf1r/temp/consensus was created


In [6]:
calculate_EFs(w_dir, docking_library)

In [ ]:
dfs = []
for root, dirs, files in os.walk('/home/mario/CADD22/'):
    for file in files:
        if file == 'enrichement_factors.csv':
            file_path = os.path.join(root, file)
            print(root)
            df = pd.read_csv(file_path)
            df = df.rename(columns={df.columns[0]: 'Method'})
            df = df.rename(columns={'EF1%': root.replace('/home/tony/CADD22/', '').replace('wocondock_performance_', '').replace('/temp/consensus', '')})
            dfs.append(df)
merged_df = pd.concat(dfs, axis=0, ignore_index=True)
merged_df = merged_df.groupby('Method', as_index=False).sum()
display(merged_df)
merged_df.to_csv('/home/tony/CADD22/results.csv', index=None)